In [1]:
import chromadb

In [1]:
import pandas as pd,psycopg2,sqlalchemy

In [2]:
from sqlalchemy import create_engine,text as to_sql_text


db_username = 'myuser'
db_password = 'root'
db_host = 'localhost'
db_port = '5432'
db_name = 'medwell_db'

connection_string = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)
conn=engine.connect()

In [4]:
db = chromadb.PersistentClient(path="user_report_data")

In [5]:
# so when user enters a report also at another server add it to the db with email as its collection name

In [ ]:
# select * from patient_report
# where user_id=69
# order by submitted_at desc
# limit 1;

In [42]:
df=pd.read_sql_query(sql=to_sql_text('''
select date_of_collection,report_type,summary,report_data from patient_report p join
patient_reportdetail r on p.id=r.report_id
where user_id=69
order by submitted_at desc
limit 1;
    '''),con= conn)

In [44]:
df

,date_of_collection,report_type,summary,report_data
0,25/3/22,blood_report,Leucopenia is observed. Packed Cell Volume (PC...,"{'pcv': {'max': 50, 'min': 36, 'unit': '%', 'v..."


In [17]:
collection=db.get_or_create_collection("rohit69")

In [30]:
collection.add(
    documents=["doc6", "doc5", "doc4",],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ],
    ids=["id4", "id5", "id6",]
)

In [33]:
collection.get(limit=5)[]

{'ids': ['id1', 'id2', 'id3', 'id4', 'id5'],
 'embeddings': None,
 'metadatas': [{'chapter': '3', 'verse': '16'},
  {'chapter': '3', 'verse': '5'},
  {'chapter': '29', 'verse': '11'},
  {'chapter': '3', 'verse': '16'},
  {'chapter': '3', 'verse': '5'}],
 'documents': ['lorem ipsum...', 'doc2', 'doc3', 'doc6', 'doc5'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [34]:
collection.count()

6

In [39]:
collection.delete(ids=[collection.get(limit=5)["ids"][0]])

In [40]:
collection.get()

{'ids': ['id2', 'id3', 'id4', 'id5', 'id6'],
 'embeddings': None,
 'metadatas': [{'chapter': '3', 'verse': '5'},
  {'chapter': '29', 'verse': '11'},
  {'chapter': '3', 'verse': '16'},
  {'chapter': '3', 'verse': '5'},
  {'chapter': '29', 'verse': '11'}],
 'documents': ['doc2', 'doc3', 'doc6', 'doc5', 'doc4'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [43]:
def generate_report_text(report_data):
    report_text = ""
    for test, details in report_data.items():
        min_val = details['min']
        max_val = details['max']
        unit = details['unit']
        value = details['value']
        if value==-1:continue
        
        value_text = f"{value} {unit}" if value != -1 else "Not available"
        
        report_text += f"{test.capitalize()}: Normal range {min_val}-{max_val} {unit}, Current value: {value_text}\n"
    
    return report_text

In [45]:
df["report_data"]=df.report_data.apply(generate_report_text)

In [46]:
df

,date_of_collection,report_type,summary,report_data
0,25/3/22,blood_report,Leucopenia is observed. Packed Cell Volume (PC...,"Pcv: Normal range 36-50 %, Current value: 43.7..."


In [50]:
text=f'''
Date of report: {df.iloc[0]["date_of_collection"]}
Report Type: {df.iloc[0]["report_type"]}
Summary of report: {df.iloc[0]["summary"]}
Report data : {df.iloc[0]["report_data"]}
'''
ly high. Bilirubin Total is in Phosphorus level.\nReport data : Pcv: Normal range 36-50 %, Current value: 43.7 %\nSodium: Normal range 135-145 mmol/L, Current value: 129.9 mmol/L\nAlbumin: Normal range 3.5-5.0 g/dL, Current value: 4.1 g/dL\nCalcium: Normal range 8.5-10.2 mg/dL, Current value: 8.86 mg/dL\nChloride: Normal range 98-107 mmol/L, Current value: 95.5 mmol/L\nProteins: Normal range 6.0-8.0 g/dL, Current value: 6.51 g/dL\nPotassium: Normal range 3.5-5.0 mmol/L, Current value: 3.59 mmol/L\nRbc_count: Normal range 4.2-5.4 million cells/μL, Current value: 4.93 million cells/μL\nWbc_count: Normal range 4.5-11.0 thousand cells/μL, Current value: 3.44 thousand cells/μL\nBlood_urea: Normal range 7-20 mg/dL, Current value: 11.2 mg/dL\nHemoglobin: Normal range 12.1-15.5 g/dL, Current value: 13.8 g/dL\nPhosphorus: Normal range 2.5-4.5 mg/dL, Current value: 4.56 mg/dL\nLymphocytes: Normal range 20-45 %, Current value: 1.68 %\nNeutrophils: Normal range 40-75 %, Current value: 1.36 %\nS_uric_acid: Normal range 3.5-7.2 mg/dL, Current value: 3.22 mg/dL\nS_creatinine: Normal range 0.6-1.2 mg/dL, Current value: 0.55 mg/dL\nS_phosphorus: Normal range 2.5-4.5 mg/dL, Current value: 4.56 mg/dL\nFasting_sugar: Normal range 70-100 mg/dL, Current value: 80.0 mg/dL\nPlatelet_count: Normal range 150000-450000 cells/μL, Current value: 290.0 cells/μL\nBilirubin_total: Normal range 0.1-1.2 mg/dL, Current value: 0.95 mg/dL\nBilirubin_direct: Normal range 0.0-0.3 mg/dL, Current value: 0.27 mg/dL\nBilirubin_indirect: Normal range 0.1-0.8 mg/dL, Current value: 0.68 mg/dL\n\n

In [52]:
text

'\nDate of report: 25/3/22\nReport Type: blood_report\nSummary of report: Leucopenia is observed. Packed Cell Volume (PCV) is 43.70% which is slightly high. Bilirubin Total is 0.95 mg/dL which is within normal range. Albumin is 4.10 g/dL which is within normal range. There is a slight increase in Phosphorus level.\nReport data : Pcv: Normal range 36-50 %, Current value: 43.7 %\nSodium: Normal range 135-145 mmol/L, Current value: 129.9 mmol/L\nAlbumin: Normal range 3.5-5.0 g/dL, Current value: 4.1 g/dL\nCalcium: Normal range 8.5-10.2 mg/dL, Current value: 8.86 mg/dL\nChloride: Normal range 98-107 mmol/L, Current value: 95.5 mmol/L\nProteins: Normal range 6.0-8.0 g/dL, Current value: 6.51 g/dL\nPotassium: Normal range 3.5-5.0 mmol/L, Current value: 3.59 mmol/L\nRbc_count: Normal range 4.2-5.4 million cells/μL, Current value: 4.93 million cells/μL\nWbc_count: Normal range 4.5-11.0 thousand cells/μL, Current value: 3.44 thousand cells/μL\nBlood_urea: Normal range 7-20 mg/dL, Current value:

In [54]:
collection.get()

5

In [55]:
collection.get(limit=5)["ids"][0]

'id2'

In [63]:
str(int("2")+5)

'7'

In [64]:
collection.add(ids=[str(int("2")+5)],documents=[text])

In [67]:
data=collection.get()

In [66]:
# chatbot thing

In [70]:
data="\n".join(data["documents"])

In [73]:
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()
client=Groq(api_key=os.environ['GROQ'])

In [106]:
question="what is the cholestrol level"

In [107]:
chat_completion = client.chat.completions.create(
    messages=[
         {"role":"system", "content":"""
         You are task is to answer questions based on the context only.Give me the reponse in json format only with two keys
         "ans": the answer in short about 60 to 70 words based on the context only. paraphrase the question while answering
         "status": value should be "true" if the answer is found in the context else will be "false".
         give me the response in json format with no preamble.
         Context: {context}
        """.format(context=data)
         },
         {
            "role": "user",
            "content": question,
        }
    ],
    model="llama3-70b-8192",
)

resp=chat_completion.choices[0].message.content

In [108]:
eval(resp)["ans"]

'There is no information about cholesterol level in the given blood report.'

In [98]:
"\n".join(collection.get()["documents"])

'doc6\ndoc5\ndoc4'

In [90]:
collection=db.create_collection("abc")

In [91]:
collection.add(documents=["doc6", "doc5", "doc4",],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ],
    ids=["4", "5", "6",])

In [95]:
collection.get(limit=1)["ids"][0]

'4'

In [3]:
def connect_db():

    connection = psycopg2.connect(
        host='localhost',
        port='5432',
        dbname='medwell_db' ,
        user='myuser' ,
        password='root' 
    )
    return connection,connection.cursor()

In [4]:
conn,cursor=connect_db()

In [8]:
cursor.execute("select email from authentication_customuser where id=69")

In [9]:
cursor.fetchone()[0]

'rohit@gmail.com'